In [196]:
import pandas as pd
import numpy as np
import os
os.chdir('/home/jovyan/work')
import sys
import init
import struct
from collections import namedtuple

import importlib
importlib.reload(do_calibration)

%matplotlib inline
plt.style.use('seaborn')
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
fileContent = None
with open(init.photometrydir+"/phot000001.pht", mode='rb') as file: # b is important -> binary
    fileContent = file.read()

In [195]:
MAGIC_STRING = "C-Munipack photometry file\r\n" # length is 28
print('size of file', len(fileContent))
############################### HEADER ###############################################
PhotHeader = namedtuple('PhotHeader', 'magic, revision, headerlen, width, height, jd, filter, exptime, ccdtemp, origin, year, mon, mday, hour, min, sec, \
                  range0, range1, gain, rnoise, fwhm_exp, fwhm_mean, fwhm_err, threshold, sharpness0, sharpness1, roundness0, roundness1, matched, match_rstars, \
                  match_istars, match_mstars, match_clip, offset0, offset1, objectdesig, objectra, objectdec, locationdesignation, \
                   locationlon, locationlat, trafoxx, trafoxy, trafox0, trafoyx, trafoyy, trafoy0 \
                  ')
headerformat = "<"+str(len(MAGIC_STRING))+"sxxxx4id70sdd70sH5Bxddddddddddddiiii3d70sdd70sdddddddd"
headerlength = struct.calcsize(headerformat)
print("header length is", headerlength)
unp = struct.unpack(headerformat, fileContent[:headerlength])
photheader = PhotHeader._make(unp)
#for name, entry in phot._asdict().items():
#    print(f"{name}: {entry}")
############################### WCS ###############################################
wcsformat= "<i" 
wcssize = struct.calcsize(wcsformat)
wcsdatalength = struct.unpack("<i", fileContent[headerlength:headerlength+wcssize])[0]
wcslength = headerlength + wcssize + wcsdatalength
print(f"wcs data length: {wcsdatalength}")
############################### APERTURES ###############################################
# typedef struct _CmpackPhtAperture
# {
# 	int id;								/**< Aperture identifier */
# 	double radius;						/**< Radius in pixels */
# } CmpackPhtAperture;

apertureformat1 = "<i"
ap1length = struct.calcsize(apertureformat1)
start = wcslength
napertures = struct.unpack(apertureformat1, fileContent[start:start+ap1length])[0]
print("napertures:", napertures)
apertureformat2 = "<" + napertures * "id"
ap2length = struct.calcsize(apertureformat2)
start = start+ap1length
apertures = struct.unpack(apertureformat2, fileContent[start:start+ap2length])
start = start + ap2length
print("apertures (nr, radius):", apertures)

############################### Stars ###############################################

# read nr of stars
nrstars = struct.unpack("i", fileContent[start:start+4])[0]
print("Number of stars:", nrstars)
start = start+4
# read stars
starformat = "<2i5d"
starsize = struct.calcsize(starformat)
stars = [] 
Star = namedtuple('Star', 'id, ref_id, x, y, skymed, skysig, fwhm')
for star in range(nrstars):
    stars.append(Star._make(struct.unpack(starformat, fileContent[start:start+starsize])))
    start = start + starsize
print(f"Read {len(stars)} stars, first 10: {stars[0:10]}")

############################## Data



magdata = []
dataformat = "<iii"
dataformatsize = struct.calcsize(dataformat)
starsleft = (len(fileContent) - start)/(dataformatsize*napertures)
print("starting at ", start, "so we can read", starsleft)
StarData = namedtuple('StarData', 'mag, err, code')
INT_MAX = 2147483647
for stentry in range(int(starsleft)):
    result = []
    for apentry in range(napertures):
#         try:
        mag, err, code = struct.unpack(dataformat, fileContent[start:start+dataformatsize])
        if mag != INT_MAX:
            mag = mag / 0x1000000
        else:
            mag = sys.float_info.max
        if err != INT_MAX:
            err = err / 0x1000000
        if code != 0 and mag != sys.float_info.max:
            print("Error code:", code, mag) # see cmpack_common.h, enum CmpackError
        result.append(StarData._make((mag, err, code)))
        start = start+dataformatsize
#         except:
#             print(stentry, apentry, start)
#             print(dataformatsize)
    magdata.append(result)
            
print(f"Read {len(magdata)} data for stars, first star: {magdata[0]}")
print(len(magdata[0]))
print("size in memory is:", (sys.getsizeof(magdata)+sys.getsizeof(stars))/1024)
        

# apertureformat1 = "<"
# wcslength = struct.calcsize(apertureformat1)
# start = headerlength+wcslength+offset
# print(wcslength)
# struct.unpack(apertureformat1, fileContent[start:start+wcslength])

# PhotWCS = namedtuple('PhotWCS', 'length, data')
# wcsformat = "<i"
# wcslength = struct.calcsize(wcsformat)
# print("length is", headerlength)
# unp = struct.unpack(headerformat, fileContent[:headerlength])
# phot = Phot._make(unp)

############################## Table of measurements

size of file 1920732
header length is 576
wcs data length: 0
napertures: 12
apertures (nr, radius): (1, 2.0, 2, 2.73, 3, 3.82, 4, 5.27, 5, 7.09, 6, 9.27, 7, 11.82, 8, 14.73, 9, 18.0, 10, 21.64, 11, 25.64, 12, 30.0)
Number of stars: 10000
Read 10000 stars, first 10: [Star(id=1, ref_id=-1, x=709.1254209922034, y=705.5452846272434, skymed=755.470179324293, skysig=36.104551065020175, fwhm=4.016703185372335), Star(id=2, ref_id=-1, x=454.7935652188418, y=534.6577074787103, skymed=750.2409284458048, skysig=35.21220875968372, fwhm=2.8302691722922146), Star(id=3, ref_id=-1, x=626.8003122294699, y=297.7888830678162, skymed=749.2679414557952, skysig=33.6410706354276, fwhm=2.7902355743683946), Star(id=4, ref_id=-1, x=1273.6085991557884, y=690.7806748180285, skymed=758.0184933569424, skysig=38.53053069856531, fwhm=2.4541505942408244), Star(id=5, ref_id=-1, x=472.7846154141154, y=707.2148446684147, skymed=746.6244341470265, skysig=34.29871022893687, fwhm=2.6527978295904733), Star(id=6, ref_id=-1, x=